# Adaptive PDE discretizations on cartesian grids 
## Volume : Divergence form PDEs
## Part : Linear elasticity
## Chapter : Elastic energy

We present a new discretization of the Dirichlet energy arising in linear elasticity, defined by a field of positive definite Hooke tensors, and associated to any small displacement field. The approach is based on Selling's decomposition of the Hooke tensor, and for this reason is only applies in dimension two at the moment.

The Dirichlet energy of linear elasitcity, defined for (small) displacement maps $v : \Omega \to R^d$, reads
$$
    E(v) := \frac 1 2 \int_\Omega \sum_{ijkl} c_{ijkl}(x) \epsilon_{ij}(x) \epsilon_{kl}(x) \ dx,
$$
where the indices $i,j,k,l$ range from $0$ to $d-1$. We denoted by $c_{ijkl}(x)$ the Hooke tensor at a point $x \in \Omega$, and introduced the symmetrized gradient of the displacement field, also known as the strain tensor $\epsilon$
$$
    \epsilon_{ij}(x) = \frac 1 2 \Big (\frac{\partial v_i}{\partial x_j} + \frac{\partial v_j}{\partial x_i} \Big).
$$

**Non staggered grids.** Finite difference schemes based on staggered grids are popular for isotropic equations, and in particular for the elastic wave equation with an isotropic Hooke tensor. In contrast, the approach presented in this notebook may deal with arbitrary anisotropy, but cannot take advantage of staggered grids.

**Second order scheme.** The numerical scheme presented in this notebook is second order accurate in space. Substantial modifications are required to implement a fourth order scheme in space.

[**Summary**](Summary.ipynb) of volume Divergence form PDEs, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1. Decomposition of a hooke tensor](#1.-Decomposition-of-a-hooke-tensor)
    * [1.1 Generic tensor](#1.1-Generic-tensor)
    * [1.2 Isotropic tensor](#1.2-Isotropic-tensor)
    * [1.3 VTI tensor](#1.3-VTI-tensor)
  * [2. Finite difference energy](#2.-Finite-difference-energy)
    * [2.1 CFL condition](#2.1-CFL-condition)
    * [2.2 Comparison with automatic differentiation](#2.2-Comparison-with-automatic-differentiation)
    * [2.3 Structure of the mass matrix](#2.3-Structure-of-the-mass-matrix)
    * [2.4 Isotropic scheme](#2.4-Isotropic-scheme)
  * [3. Three dimensions](#3.-Three-dimensions)
    * [3.1 Decomposition of a single tensor](#3.1-Decomposition-of-a-single-tensor)
    * [3.2 Structure of the sparse matrix](#3.2-Structure-of-the-sparse-matrix)



**Acknowledgement.** The experiments presented in these notebooks are part of ongoing research, 
some of it with PhD student Guillaume Bonnet, in co-direction with Frederic Bonnans, 
and PhD student François Desquilbet, in co-direction with Ludovic Métivier.

Copyright Jean-Marie Mirebeau, Centre Borelli, ENS Paris-Saclay, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"..") # Allow import of agd from parent directory (useless if conda package installed)
#from Miscellaneous import TocTools; print(TocTools.displayTOC('ElasticEnergy','Div'))

In [2]:
from agd import LinearParallel as lp
from agd import FiniteDifferences as fd
from agd.Metrics.Seismic import Hooke
from agd import AutomaticDifferentiation as ad
from agd import Domain
from agd.Plotting import savefig; #savefig.dirName = 'Images/ElasticityDirichlet'

norm_infinity = ad.Optimization.norm_infinity
norm_average = ad.Optimization.norm_average
mica,_ = Hooke.mica # Hooke tensor associated to this crystal

In [3]:
import numpy as np; xp=np; allclose=np.allclose
import matplotlib.pyplot as plt
from copy import copy
import scipy.sparse; import scipy.sparse.linalg
import itertools

In [4]:
def ReloadPackages():
    from Miscellaneous.rreload import rreload
    global lp,fd,Hooke,ad,Domain
    lp,fd,Hooke,ad,Domain = rreload([lp,fd,Hooke,ad,Domain],rootdir="../..")
    ad.array.caster = xp.asarray

### 0.1 Additional configuration

Uncomment the following line to run the notebook on the GPU. (This is purely for compatibility testing, as no intensive computation is involved.)

In [5]:
#xp,mica,allclose = map(ad.cupy_friendly,(xp,mica,allclose))

## 1. Decomposition of a hooke tensor

A Hooke tensor defines a quadratic form on the set of symmetric matrices $\epsilon \in S_d$
$$
    c(\epsilon,\epsilon) = \sum_{ijkl} c_{ijkl} \epsilon_{ij} \epsilon_{kl}.
$$
Note that $S_d$ has dimension $D=d (d+1)/2$. We limit our attention to the case $d=2$, since the case $d=1$ is excessively trivial, and the case $d=3$ would require an implementation of the $6$-dimensional Voronoi reduction, which we do not have at this stage.

We use Selling's decomposition to rewrite this quadratic form as
$$
    c(\epsilon,\epsilon) = \sum_r \rho_r \mathrm{Tr}(\epsilon m_r)^2
$$
where $\rho_r \geq 0$, $m_r \in S_2(Z)$ is a symmetric matrix with integer coordinates, and $0 \leq r < D (D+1)/2=6$. For that purpose, we rely on Selling's decomposition of the Hooke tensor, which applies in dimension since the linear space of $2\times 2$ symmetric matrices has dimension $3$.

<!--- In the following, we denote as usual $c(\epsilon) := c(\epsilon,\epsilon)$. --->

The stress tensor $\sigma$ depends linearly on the strain tensor $\epsilon$, for a given hooke tensor $c$, and is characterized by the identity
$$
    \mathrm{Tr}(\sigma \epsilon) = c(\epsilon,\epsilon).
$$
With the correct index conventions, one has $\sigma_{ij} = \sum_{kl} c_{ijkl} \epsilon_{kl}$, or simply $\sigma = c \epsilon$.

### 1.1 Generic tensor

We illustrate the decomposition on a generic tensor, describing the anisotropic elasticity of a mica rock medium, whose layers are rotated.

In [6]:
hooke = mica.extract_xz().rotate_by(0.5)

Selling's decomposition involves $D=6$ weights and offsets, in dimension $d=2$. 

In [7]:
coefs,moffsets = hooke.Selling()

In [8]:
coefs

array([ 4.39753907, 31.3882811 ,  3.86972663, 17.65302611, 41.16677899,
       29.81855447])

The offsets are presented as symmetric matrices, with integer entries.

In [9]:
moffsets.shape

(2, 2, 6)

In [10]:
for i in range(6): print(moffsets[...,i],"\n")

[[1 1]
 [1 1]] 

[[-1 -1]
 [-1  0]] 

[[2 1]
 [1 1]] 

[[ 0  0]
 [ 0 -1]] 

[[-1  0]
 [ 0  0]] 

[[1 0]
 [0 1]] 



In order to check the correctness of the decomposition, let us introduce an arbitrary $2\times 2$ symmetric matrix, and evaluate $c(m)$ using the original expression and the decomposition.

In [11]:
m = xp.array(((1.5,2.3),(2.3,3.9)))

In [12]:
sum_hooke = hooke.dot_AA(m) 

In [13]:
mm = fd.as_field(m,coefs.shape)
sum_inner = (coefs*lp.trace(lp.dot_AA(mm,moffsets))**2).sum(axis=0)

In [14]:
assert allclose(sum_hooke,sum_inner)

### 1.2 Isotropic tensor

Isotropic elasticity tensors only have two degrees of freedom. Without loss of generality, we consider the Lamé parameters. These parameters relate the strain tensor $\epsilon$ with the stress tensor $\sigma$
$$
    \sigma = 2 \mu \epsilon + \lambda \mathrm{Tr}(\epsilon) \mathrm{Id},
$$
and the quadratic form reads 
$$
    c(\epsilon,\epsilon) = 2 \mu \mathrm{Tr}(\epsilon^2) + \lambda \mathrm{Tr}(\epsilon)^2.
$$

In [15]:
hooke = Hooke.from_Lame(xp.array(1.),2.) # xp.array is a type hint for GPU
print(f"""An isotropic hooke tensor :\n{hooke.hooke}\n""")

An isotropic hooke tensor :
[[5. 1. 0.]
 [1. 5. 0.]
 [0. 0. 2.]]



In [16]:
print(f"""Isotropic Hooke tensors are linear combinations of: \n{Hooke.from_Lame(1.,0.).hooke}\n"""
      f"""and \n{Hooke.from_Lame(0.,1.).hooke}\n""")

Isotropic Hooke tensors are linear combinations of: 
[[1. 1. 0.]
 [1. 1. 0.]
 [0. 0. 0.]]
and 
[[2. 0. 0.]
 [0. 2. 0.]
 [0. 0. 1.]]



As their name suggests, isotropic hooke tensors are invariant under rotations.

In [17]:
hooke_rot = copy(hooke).rotate_by(0.5)
assert allclose(hooke.hooke,hooke_rot.hooke)

Selling's decomposition of an isotropic Hooke tensor is very structured and predictable. It involves offsets $m_r$ which are independent of the parameters $(\lambda,\mu)$, and weights $\rho_r(\lambda,\mu)$ depending linearly on the Lame parameters. In addition, several of these coefficients vanish.

In [18]:
coefs,moffsets = hooke.Selling()

In [19]:
coefs

array([-0., -0.,  2.,  4.,  4.,  1.])

In [20]:
for c,o in zip(coefs,np.moveaxis(moffsets,-1,0)): 
    if c!=0.:
        print(f"coef:{c}, moffset:\n{o}\n")

coef:2.0, moffset:
[[0 1]
 [1 0]]

coef:4.0, moffset:
[[ 0  0]
 [ 0 -1]]

coef:4.0, moffset:
[[-1  0]
 [ 0  0]]

coef:1.0, moffset:
[[1 0]
 [0 1]]



### 1.3 VTI tensor

A VTI Hooke tensor, or slightly more generally a tetragonal Hooke tensor, has a block diagonal form.
One of the blocks has size $d \times d$, and is fully generic in dimension $d=2$.

In [21]:
hooke = Hooke.from_tetragonal(1.,2.,3.,4.,5.,6.)

In [22]:
hooke.extract_xz().hooke

array([[1., 3., 0.],
       [3., 4., 0.],
       [0., 0., 5.]])

In [23]:
hooke.hooke

array([[1., 2., 3., 0., 0., 0.],
       [2., 1., 3., 0., 0., 0.],
       [3., 3., 4., 0., 0., 0.],
       [0., 0., 0., 5., 0., 0.],
       [0., 0., 0., 0., 5., 0.],
       [0., 0., 0., 0., 0., 6.]])

## 2. Finite difference energy

We approximate the linear elastic energy using a second order accurate finite differences scheme, which exploits our tensor decomposition. The scheme is based on the identity
$$
    c(\epsilon,\epsilon) = \sum_r \rho_r \mathrm{Tr}(m_r \nabla v)^2
$$
where $(\rho_r,m_r)$ is Selling's decomposition of $c$. We could replace $\sigma$ with $\nabla v$ thanks to the symmetry of $m_r$.

Then we use the finite difference approximations
$$
    \mathrm{Tr}(m \nabla v) = \sum_{0 \leq i < d} \frac{v_i(x+h s_i m[i] )-v_i(x)}{h s_i} + O(h),
$$
where $s_i\in\{-1,1\}$, $0\leq i < d$ are arbitrary signs, and $m[i]$ denotes the $i$-th column of $m$ (which is a symmetric matrix). Squaring this expression, averaging over all possible sign choices, and summing with weights $\rho_r$, we obtain a second order consistent approximation of the local linear elastic energy.
$$
    c(\epsilon,\epsilon) = \sum_{0 \leq r \leq D (D+1)/2} \frac{\rho_r}{2^d} \sum_{s \in \{-1,1\}^d}  
    \Big(\sum_{0 \leq i < d} \frac{v_i(x+h s_i m_r[i] )-v_i(x)}{h s_i}\Big)^2.
$$

**Remark**
If the coordinates of $m_r[i]$ are not co-prime, for some $0 \leq r < D (D+1)/2$ and $0 \leq i < d$, then one can improve the scheme taking advantage of this fact in the finite differences.

**Scheme order.**
For symmetry reasons, using first order finite differences yields a second order estimation of the elastic energy. 
Likewise, using third order finite differences would yield a fourth order estimation of the elastic enery, but this does not seem practical.

In [24]:
def ElasticEnergy(v,hooke,dom):
    """
    Finite differences approximation of c(ϵ,ϵ), where c is the Hooke tensor and ϵ the strain tensor,
    which is twice the (linearized) elastic energy density.
    """
    d=len(v)
    coefs,moffsets = hooke.Selling()
    dvp = tuple( dom.DiffUpwind(v[i], moffsets[i]) for i in range(d))
    dvm = tuple(-dom.DiffUpwind(v[i],-moffsets[i]) for i in range(d))
    
    # Consistent approximations of Tr(moffset*grad(v))
    dv = ad.array([sum(s) for s in itertools.product(*zip(dvp,dvm))])
    dv2 = (dv**2).sum(axis=0)
    
    coefs = fd.as_field(coefs,v.shape[1:]) * 2**(-d)
    return (coefs*dv2).sum(axis=0) 

### 2.1 CFL condition

The above discretization of the elastic energy is intended to be used in numerical simulations of the elastic wave equation, which involves a CFL condition. See the [discussion in the one dimension case](Time1D_Div.ipynb). The key to deriving this condition is to upper bound the quadratic form $c(\epsilon,\epsilon)$ in terms of a quadratic form with only diagonal coefficients.

Recall the Cauchy-Schwartz inequality, or arithmetico-geometric inequality
$$
    \Big(\sum_{1 \leq k \leq K} a_k\Big)^2 \leq k \sum_{1 \leq k \leq K} a_k^2,
$$
and observe that $c(\epsilon,\epsilon)$ is a sum of squares, each comprising $K = 2 d$ terms. Denoting by $A$ the sparse matrix associated with the integrated elastic energy, and by $D$ its diagonal, one therefore has $A \preceq K D$.

In the case where the hooke tensors are constant, one may be slightly more explicit, observing that 
$$
    \mathrm{Tr}(c) = \sum_r \rho_r \mathrm{Tr}(m_r^2) \geq \sum_r \rho_r,
$$
since the matrices $m_r$ have integer coefficients (and are non-zero). By $\mathrm{Tr}(c)$ we denote the trace of the Mandel form of $c$, which is also its trace as a quadratic form on the space of $2\times 2$ symmetric matrices equipped with the Frobenius inner product.
From this point, one gets $A \leq L \mathrm{Tr}(c) \mathrm{Id}$, where $L = 2 K d = (2d)^2$.

In [25]:
hooke = mica.extract_xz().rotate_by(0.5)
coefs,moffsets = hooke.Selling()

In [26]:
tr_decomp = np.sum(coefs*lp.trace(lp.dot_AA(moffsets,moffsets)))
tr_Mandel = lp.trace(hooke.to_Mandel())
assert allclose(tr_decomp,tr_Mandel)

### 2.2 Comparison with automatic differentiation

For comparison, we also evaluate the elastic energy using automatic differentiation for the derivatives of $v$, instead of finite differences, which yields an exact expression. This is only possible when $v$ is provided as a differentiable function, rather than a table.

In [27]:
def ElasticEnergy_ad(v,hooke,X):
    """
    Returns c(ϵ,ϵ), where c is the Hooke tensor, and ϵ is the stress tensor.
    The displacement field must be provided as a function, compatible with automatic differentiation.
    """
    # Differentiate the displacement field
    X_ad = ad.Dense.identity(constant=X,shape_free=(2,))
    grad = v(X_ad).gradient()
    
    # Compute the stress tensor, and the inner product associated with the Hooke tensor.
    ϵ = 0.5*(grad+lp.transpose(grad))
    return hooke.dot_AA(ϵ) 

Let us observe the convergence of the finite element energy toward the exact energy in a smooth periodic setting.

In [28]:
def v(X):
    x0,x1 = X*(2.*np.pi)
    return ad.array((np.cos(x0) - 2.*np.sin(x1),np.cos(x0+2*x1)))

def hooke(X):
    x0,x1 = X*(2.*np.pi)
    angle = 0.3*np.sin(x0)+0.5*np.cos(x1)    
    return mica.extract_xz().rotate_by(angle)

In [29]:
n=20
aX,h = xp.linspace(0,1,n,endpoint=False,retstep=True)
X=ad.array(np.meshgrid(aX,aX,indexing='ij'))
dom = Domain.MockDirichlet(X.shape,h,padding=None) #Periodic domain (wrap instead of pad)

In [30]:
energy_density_fd = ElasticEnergy(   v(X),hooke(X),dom) # Uses samples of v
energy_density_ad = ElasticEnergy_ad(v,   hooke(X),X)   # Uses function v

The finite differences scheme is second order accurate. A fourth order scheme could easily be achieved, in the domain interior, by using higher order finite differences. The treatment of non-periodic boundary conditions in another story altogether, that will be addressed later.

In [31]:
norm_infinity(energy_density_fd-energy_density_ad) / norm_average(energy_density_ad)

0.7490743389566975

In order to obtain the total elastic energy, the density must be integrated over the domain, not forgetting about the $1/2$ factor.

In [32]:
energy_ad = 0.5 * energy_density_ad.sum() * h**2
energy_fd = 0.5 * energy_density_fd.sum() * h**2
print(f"Relative error: {(energy_ad-energy_fd)/energy_fd}")

Relative error: 0.03070905470908115


### 2.3 Structure of the mass matrix

We evaluate the finite difference scheme on a second order sparse AD vector, to find the mass matrix of the elastic energy. At each point in the domain, the energy density depends on a number of neighbor values of the strain tensor, referred to as the stencil.

In [33]:
v_sp = ad.Sparse2.identity(constant=np.zeros_like(X))
energy_density_sp = ElasticEnergy(v_sp,hooke(X),dom)
print(f"Stencil cardinality: {energy_density_sp.size_ad2}")

Stencil cardinality: 384


The *simplification* step compresses the sparse matrix by merging entries corresponding to the same coefficient, and removing zero coefficients.
Note that we use `atol=0.` to also remove coefficients arising from cancellation effects: the sum of merged entries vanishes. 
A more careful implementation of `Energy_fd` may allow to bypass these simplification steps.

In [34]:
energy_density_sp.simplify_ad(atol=True,rtol=True)
print(f"Stencil cardinality: {energy_density_sp.size_ad2}")

Stencil cardinality: 70


The energy density at a given point in space is given by a sparse quadratic form, with the above number of non-zero entries at most.

In [35]:
energy_density_sp[9,5].triplets()

(array([ 5.07960333e+03, -5.07960333e+03,  2.43309559e+03,  1.06706077e+02,
        -1.06706077e+02, -2.43309559e+03,  6.51713229e+04, -6.51713229e+04,
         3.27025834e+03, -3.27025834e+03,  3.27501189e+02, -3.27501189e+02,
         1.63750594e+02, -1.63750594e+02, -5.07960333e+03, -6.51713229e+04,
        -3.27501189e+02,  1.41156855e+05, -3.27501189e+02, -6.51713229e+04,
        -5.07960333e+03, -3.27501189e+02,  3.27501189e+02, -1.63750594e+02,
         1.63750594e+02, -6.51713229e+04,  6.51713229e+04, -3.27025834e+03,
         3.27025834e+03, -5.07960333e+03,  5.07960333e+03, -2.43309559e+03,
        -1.06706077e+02,  1.06706077e+02,  2.43309559e+03,  2.43309559e+03,
        -2.43309559e+03,  4.86619118e+03, -4.86619118e+03,  1.06706077e+02,
         1.63750594e+02, -1.63750594e+02, -1.06706077e+02,  5.40913343e+02,
        -5.40913343e+02,  3.27025834e+03, -3.27025834e+03,  2.13139514e+04,
        -2.13139514e+04, -4.86619118e+03, -5.40913343e+02, -2.13139514e+04,
         5.3

In [36]:
np.max(np.min(np.abs(energy_density_sp.coef2),axis=-1))

2465.644648535162

Let us construct the sparse symmetric matrix associated to the total energy.

In [37]:
energy_sp = 0.5 * energy_density_sp.sum() * h**2
energy_hess = energy_sp.hessian_operator()

The number of non-zero entries for each line is approximately 20.

In [38]:
energy_hess.nnz / energy_hess.shape[0]

20.755

Since the finite difference energy is quadratic, it is exactly reproduced by the quadratic form defined by is half hessian.

In [39]:
v_fl = v(X).flatten()
energy_fl = 0.5*np.dot(v_fl,energy_hess*v_fl)

In [40]:
assert allclose(energy_sp.as_func(v_fl),energy_fd)

In [41]:
# Fails on cupy 6., works on cupy 7.4 (bug in cupyx sparse matrices)
assert allclose(energy_fl,energy_fd)

### 2.4 Isotropic scheme

When the material is isotropic, the numerical scheme simplifies substantially. 

**Staggered grid discretization.** For isotropic materials, there exists scheme taking advantage of staggered grids, which are expected to be more accurate than the one proposed in this notebook.

In [42]:
v_sp = ad.Sparse2.identity(constant=np.zeros_like(X))
energy_density_sp = ElasticEnergy(v_sp,Hooke.from_Lame(xp.array(1.),2.),dom)
print(f"Stencil cardinality: {energy_density_sp.size_ad2} (before simplification)")
energy_density_sp.simplify_ad(atol=True,rtol=True)
print(f"Stencil cardinality: {energy_density_sp.size_ad2} (after simplification)")
energy_hess = energy_density_sp.hessian_operator()
print(f"Stencil cardinality: {energy_hess.nnz / energy_hess.shape[0]} (linear operator)")

Stencil cardinality: 384 (before simplification)


Stencil cardinality: 42 (after simplification)
Stencil cardinality: 9.0 (linear operator)


## 3. Three dimensions

The scheme also works in three dimensions.

### 3.1 Decomposition of a single tensor

In [43]:
hooke = mica.rotate_by(0.5, (1,2,3)) # Rotate around some arbitrary axis

In [44]:
coefs,moffsets = hooke.Selling()

In [45]:
print("Coefficients ρ_i : ",coefs)
print("Matrix offsets D_i : \n",np.moveaxis(moffsets,-1,0).astype(int))

Coefficients ρ_i :  [  0.19785622   0.36125226   0.52227781   0.54082001   1.28421131
   2.63328284   2.83550192   3.29685521   3.88586822   3.95321744
   5.08269759   5.11617677   6.04766768   7.2971531    8.98090467
  10.859143    17.5386092   23.40618513  34.13150106  66.37137614
 129.73804697]
Matrix offsets D_i : 
 [[[ 0 -1 -1]
  [-1  0  1]
  [-1  1 -1]]

 [[ 1 -1 -1]
  [-1  0  1]
  [-1  1  0]]

 [[-1 -1  0]
  [-1  0  1]
  [ 0  1  0]]

 [[-1  0  1]
  [ 0  0  0]
  [ 1  0  1]]

 [[ 1 -2 -1]
  [-2  0  1]
  [-1  1  0]]

 [[ 0 -1  0]
  [-1  0  0]
  [ 0  0  1]]

 [[-1  0  0]
  [ 0 -1  0]
  [ 0  0  0]]

 [[-2  0  1]
  [ 0  0  0]
  [ 1  0  0]]

 [[ 0 -1  0]
  [-1 -1  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  1  1]
  [ 0  1  0]]

 [[ 0 -1  0]
  [-1  0  1]
  [ 0  1  0]]

 [[-1 -1  0]
  [-1 -1  0]
  [ 0  0  0]]

 [[ 0 -1  0]
  [-1  1  1]
  [ 0  1  0]]

 [[-2  0  1]
  [ 0 -1  0]
  [ 1  0  0]]

 [[-1  0  0]
  [ 0  0  0]
  [ 0  0 -1]]

 [[ 1  0 -1]
  [ 0  0  0]
  [-1  0  0]]

 [[-1  0  0]
  [ 0 -1 

In [46]:
if xp is not np: raise ad.DeliberateNotebookError(
        """End of notebook raises 'Out of memory' on cupy, reasons unknown.""")

### 3.2 Structure of the sparse matrix

We obtain a matrix fill of 15 for an isotropic material, and approximately 80 for an anisotropic material. (In the latter case, the figure will increase a bit if the material is non-constant.)

In [47]:
n=8
aX,h = xp.linspace(0,1,n,endpoint=False,retstep=True)
X=ad.array(np.meshgrid(aX,aX,aX,indexing='ij'))
dom = Domain.MockDirichlet(X.shape,h,padding=None) #Periodic domain (wrap instead of pad)

In [48]:
v_sp = ad.Sparse2.identity(constant=np.zeros_like(X))
energy_density_sp = ElasticEnergy(v_sp,Hooke.from_Lame(xp.array(1.),2.,vdim=3),dom)
print(f"Stencil cardinality: {energy_density_sp.size_ad2} (before simplification)")
energy_density_sp.simplify_ad(atol=True,rtol=True)
print(f"Stencil cardinality: {energy_density_sp.size_ad2} (after simplification)")
energy_hess = energy_density_sp.hessian_operator()
print(f"Stencil cardinality: {energy_hess.nnz / energy_hess.shape[0]} (linear operator)")

Stencil cardinality: 6048 (before simplification)


Stencil cardinality: 105 (after simplification)
Stencil cardinality: 15.0 (linear operator)


In [49]:
hooke = mica.rotate_by(0.5,(1,2,3))
v_sp = ad.Sparse2.identity(constant=np.zeros_like(X))
energy_density_sp = ElasticEnergy(v_sp,hooke,dom)
print(f"Stencil cardinality: {energy_density_sp.size_ad2} (before simplification)")
energy_density_sp.simplify_ad(atol=True,rtol=True)
print(f"Stencil cardinality: {energy_density_sp.size_ad2} (after simplification)")
energy_hess = energy_density_sp.hessian_operator()
print(f"Stencil cardinality: {energy_hess.nnz / energy_hess.shape[0]} (linear operator)")

Stencil cardinality: 6048 (before simplification)


Stencil cardinality: 369 (after simplification)
Stencil cardinality: 79.0 (linear operator)
